#Use this code when u want to focus only LR and ignore other transformation

In [ ]:
#Regression imputation for Value and MORTDUE

from sklearn.linear_model import LinearRegression

# Prepare the data
valid_data = raw_data.dropna(subset=['mort_due', 'current_property_value'])

# Train a model to predict 'current_property_value' based on 'mort_due'
model_value = LinearRegression()
model_value.fit(valid_data[['mort_due']], valid_data['current_property_value'])

# Train a model to predict 'mort_due' based on 'current_property_value'
model_mort_due = LinearRegression()
model_mort_due.fit(valid_data[['current_property_value']], valid_data['mort_due'])

# Creating columns for regression imputed values

raw_data['reg_mort_due'] = raw_data['mort_due']
raw_data['reg_current_property_value'] = raw_data['current_property_value']

# Impute missing 'current_property_value' using the regression model
missing_value = raw_data['current_property_value'].isnull() & raw_data['mort_due'].notnull()
raw_data.loc[missing_value, 'reg_current_property_value'] = model_value.predict(raw_data.loc[missing_value, ['mort_due']])

# Impute missing 'mort_due' using the regression model
missing_mort_due = raw_data['mort_due'].isnull() & raw_data['current_property_value'].notnull()
raw_data.loc[missing_mort_due, 'reg_mort_due'] = model_mort_due.predict(raw_data.loc[missing_mort_due, ['current_property_value']])

# Handle cases where both values are missing (optional adjustment may be needed)
raw_data.loc[
    raw_data['mort_due'].isnull() & raw_data['current_property_value'].isnull(),
    ['reg_mort_due', 'reg_current_property_value']
] = 0  # or some other strategy for handling entirely missing cases


In [ ]:
print(raw_data['reg_mort_due'].isnull().sum())
print(raw_data['reg_current_property_value'].isnull().sum())